In [7]:
# import packages used in this notebook
import os # need this to read the list of files in the directory
import io # need this to read the data from the zip file
from zipfile import ZipFile  # need this to read the data from the zip file
import tarfile # need this to read the data from the tar file
import json
import bz2
import gzip
import re # this is used for the regular expression to extract the subject from the file 

In [8]:
#!hadoop fs -mkdir /ca4


In [9]:
def loadtoMongo(contents, subject):
    import pymongo
    client = pymongo.MongoClient("localhost", 27017)
    db = client.twitter
    db.tweets
    for line in contents:
        try:
            
            # filter the data for my subject
            fltrData = re.search(r'\b{}\b'.format(subject), line.lower())

            if fltrData:
                try:
                    data = json.loads(line)
                    #data.show()
                    #if the subject is found we write into hadoop
                    db.tweets.insert_one(data)
                except KeyError as e:
                    #print("KeyError encountered", {e}) # error is encountered mainly due to deleted tweets 
                    #print(line)
                    continue
        except json.decoder.JSONDecodeError as e:
            print("Error encountered at line: ", {e})
            continue
    
   

In [19]:
tweetDrv = "/home/sba22230/shared-drives/E:/TwitterStream/"
import fnmatch
#tweetDrv = "./SampleTweets/"
tweetWrappers = fnmatch.filter(os.listdir(tweetDrv), 'twitter-stream-202*') # doing this to speed things up otherwise it won't be done in time

#display(tweetWrappers)


In [20]:
def bz2_func(data):
    data = bz2.decompress(data)
    data = io.StringIO(data.decode('utf-8'))
    contents = data.readlines()
    return contents

def gz_func(data):
    
    data = gzip.decompress(data)
    data = io.StringIO(data.decode('utf-8'))
    contents = data.readlines()
    return contents

for wrap in tweetWrappers:
    file = tweetDrv + wrap
    if file.endswith('.zip'):
        print("Zip file found " + wrap) # need to a count of the files in the zip file format
        #display(file)
        try:
            with ZipFile(file, 'r') as zip:
                zipLn = len(zip.namelist())
                for i in range(zipLn):
                    #print(i, zip.namelist()[i])
                    files = zip.namelist()[i]
                    if files.endswith('bz2'):
                        data = zip.read(files)
                        contents = bz2_func(data) # now that we have the contents, we can push the tweets into mongoDB    
                        loadtoMongo(contents, 'rugby')
                    elif files.endswith('gz'):
                        data = zip.read(files)
                        contents = gz_func(data)
                        loadtoMongo(contents, 'rugby')
        except: 
            continue
    elif file.endswith('.tar'):
        print("Tar file found " + wrap) # need to a count of the files in the tar file format
        try:
            with tarfile.open(file, 'r') as tar:
                tarLn = len(tar.getnames())
                for i in range(tarLn):
                    #print(i, tar.getnames()[i])
                    files = tar.getnames()[i]
                    if files.endswith('bz2'):
                        data = tar.extractfile(files)
                        contents = bz2_func(data.read()) # tar behaves differently to zip it doesn't return a bytes object    
                        loadtoMongo(contents, 'rugby')
                    elif files.endswith('gz'):
                        data = tar.extractfile(files)
                        contents = gz_func(data.read())
                        loadtoMongo(contents, 'rugby')
        except: # some tar files did not download correctly 
            continue 
        
    else:
        #print("unknown file" + os.path.splitext(file)[-1])
        continue
    os.replace(file, tweetDrv + "processed/" + wrap) # move the file to the processed folder


Zip file found twitter-stream-2021-07-21.zip
Zip file found twitter-stream-2021-07-22.zip
Zip file found twitter-stream-2021-07-24.zip
Zip file found twitter-stream-2021-07-25.zip
Zip file found twitter-stream-2021-07-26.zip
Zip file found twitter-stream-2021-07-27.zip
Zip file found twitter-stream-2021-07-28.zip
Zip file found twitter-stream-2021-07-29.zip
Zip file found twitter-stream-2021-07-30.zip
Zip file found twitter-stream-2021-07-31.zip
